In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-fulfillment/sample_submission.csv
/kaggle/input/non-fulfillment/train.csv
/kaggle/input/non-fulfillment/test.csv


In [2]:
# Kaggle Notebook에서 AutoGluon 설치 (첫 실행 시 한 번만)
!pip install -q --upgrade autogluon
!pip install --upgrade scikit-learn==1.4.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.0 MB/s eta 0:00

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import warnings
from autogluon.tabular import TabularPredictor
warnings.filterwarnings(action='ignore')

In [4]:
# 학습/평가 데이터 로드
# train_df = pd.read_csv('./train.csv').drop(columns=['UID'])
# test_df = pd.read_csv('./test.csv').drop(columns=['UID'])
# train_df = pd.read_csv('/kaggle/input/non-fulfillment/train.csv').drop(columns=['UID'])
# test_df = pd.read_csv('/kaggle/input/non-fulfillment/test.csv').drop(columns=['UID'])
# 데이터 로드 및 전처리
train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')

# UID 컬럼 제거
train_data.drop(columns=['UID'], inplace=True)
test_data.drop(columns=['UID'], inplace=True)

In [5]:
train_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0


In [6]:
# 타깃 변수 지정
label = '채무 불이행 여부'

# AutoGluon TabularPredictor 초기화 및 학습
predictor = TabularPredictor(label=label, eval_metric='roc_auc').fit(
    train_data,
    presets='best_quality',       # 높은 성능을 위한 프리셋 사용
    num_bag_folds=5,              # 교차 검증 폴드 수 증가
    num_stack_levels=2,           # 스택 앙상블 레벨 추가
    time_limit=3600               # 학습에 충분한 시간 할당 (예: 1시간)
)

# 테스트 데이터에 대해 예측 (각 클래스별 확률 예측)
preds_proba = predictor.predict_proba(test_data)
test_data['채무 불이행 확률'] = preds_proba[1]  # 1번 클래스(채무 불이행)의 확률 사용

No path specified. Models will be saved in: "AutogluonModels/ag-20250322_080956"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.29 GB / 31.35 GB (96.6%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

(_ray_fit pid=2759) [1000]	valid_set's binary_logloss: 0.55893


(_dystack pid=253) 	0.7478	 = Validation score   (roc_auc)
(_dystack pid=253) 	12.68s	 = Training   runtime
(_dystack pid=253) 	1.32s	 = Validation runtime
(_dystack pid=253) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 27.70s of the 524.69s of remaining time.
(_dystack pid=253) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.02%)
(_dystack pid=253) 	Warning: Exception caused NeuralNetTorch_r22_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=253) 		ray::_ray_fit() (pid=2924, ip=172.19.2.2)
(_dystack pid=253)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 413, in _ray_fit
(_dystack pid=253)     fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
(_dystack pid=253)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/

In [7]:
# 제출 파일 불러오기 및 예측 결과 반영
submission = pd.read_csv('/kaggle/input/non-fulfillment/sample_submission.csv')
submission['채무 불이행 확률'] = test_data['채무 불이행 확률']
submission.to_csv('submission_autogluon.csv', index=False)

print("AutoGluon을 이용한 예측 및 제출 파일 생성 완료!")

AutoGluon을 이용한 예측 및 제출 파일 생성 완료!
